In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import onnxruntime
import PIL.Image as Image
import torch
import torch.onnx
import torchvision
from mnist import ConvNet

In [ ]:
num_classes = 10

model = ConvNet(num_classes)
model.load_state_dict(torch.load('model.ckpt'))
model.eval()

dummy_input = torch.randn((1, 1, 28, 28))
torch.onnx.export(model, dummy_input, "model.onnx")

In [ ]:
x = Image.open('mnist_2.jpg', mode='r')
x_th = torchvision.transforms.ToTensor()(x).reshape(1, 1, 28, 28)
x_np = np.asarray(x_th)

In [ ]:
y_th = model(x_th)

session = onnxruntime.InferenceSession("model.onnx")
inname = [input.name for input in session.get_inputs()]
outname = [output.name for output in session.get_outputs()]
y_np = session.run(outname, {inname[0]: x_np})

plt.imshow(x)
print(f'PyTorch result = {y_th}')
print(f'ONNX result = {y_np}')